In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import os
import torch
import copy
from torch.utils.data import Dataset, DataLoader

from sklearn.externals import joblib

from functools import reduce
from util import *

In [2]:
exp_name = 'exp2'
checkpoints_path = os.path.join('../checkpoints/', exp_name)
selected_model = os.path.join(checkpoints_path, 'action_dim_25batch_size_512_drop_prob_0.0_gamma_0.9_hidden_dim_1024_lr_0.0001_num_epochs_300_num_hidden_5_option_linear_state_dim_46_target_update_10_use_scheduler_False_time_9895668.chk')

data_path = '../data'
data_dict = joblib.load(os.path.join(data_path, 'data_dict.pkl'))

In [3]:
final_df_test = pd.read_csv(os.path.join(data_path, 'test_state_action_reward_df.csv'))

In [4]:
use_gpu = torch.cuda.is_available()
print(use_gpu)

False


In [5]:
eval_dataset = evaluation_Dataset(data_dict['test']['X'], 
                                  data_dict['test']['action']
                                 )

In [6]:
eval_loader = DataLoader(eval_dataset, 
                            512,
                            shuffle = False,
                            num_workers = 32
                         )

In [7]:
def evaluate_model(model, the_loader, use_gpu = False):
    
    model.train(False)
    i = 0
    for states, actions in the_loader:
        i = i + 1
        batch_size = states.shape[0]
        
        if use_gpu:
            states, actions = Variable(states.cuda(async = True)), \
                                Variable(actions.cuda(async = True))
                
        else:
            states, actions = Variable(states), \
                                Variable(actions)
                
        outputs = model(states)
        best_actions = torch.max(outputs, 1)[1].data
        
        if use_gpu:
            # Get Q(s, a)
            current_state_values = outputs[torch.LongTensor(np.arange(batch_size).tolist()).cuda(),
                                           actions.data]
            # Get Q(s, a*)
            best_policy_values = outputs[torch.LongTensor(np.arange(batch_size).tolist()).cuda(),
                                        best_actions]
                
        else:
            # Get Q(s, a)
            current_state_values = outputs[torch.LongTensor(np.arange(batch_size)),
                                           actions.data]
            # Get Q(s, a*)
            best_policy_values = outputs[torch.LongTensor(np.arange(batch_size)),
                                                   best_actions]
            
        if i == 1:
            predicted_state_action_values_list = current_state_values
            predicted_state_action_values_list_best = best_policy_values
            best_actions_list = best_actions
        else:
            predicted_state_action_values_list = torch.cat((predicted_state_action_values_list, 
                                                            current_state_values))
            predicted_state_action_values_list_best = torch.cat((predicted_state_action_values_list_best, 
                                                            best_policy_values))
            best_actions_list = torch.cat((best_actions_list, 
                                            best_actions))
    
    return predicted_state_action_values_list, predicted_state_action_values_list_best, best_actions_list

In [8]:
# Hardcoded selected model for now
config = {
          'state_dim' : data_dict['train']['X'].shape[1],
          'action_dim' : 25,
          'gamma' : 0.9,
          'batch_size' : 512,
          'lr' : 1e-4,
          'num_epochs' : 300,
          'hidden_dim' : 1024,
          'num_hidden' : 5,
          'drop_prob' : 0.0,
          'target_update': 10,
          'option' : 'linear',
          'use_scheduler' : False
         }

In [9]:
model = dueling_net(D_in = config['state_dim'], 
                    H = config['hidden_dim'], 
                    D_out = config['action_dim'],
                    drop_prob = config['drop_prob'],
                    num_hidden = config['num_hidden'],
                    option = config['option']
                   )

if use_gpu:
    model = model.cuda()

if use_gpu:
    model.load_state_dict(torch.load(selected_model))
else:
    model.load_state_dict(torch.load(selected_model, map_location=lambda storage, loc: storage))

predicted_Q_physician, predicted_Q_optimal, best_actions_list = evaluate_model(model, 
                                                                               eval_loader, 
                                                                               use_gpu)

In [10]:
best_actions_list


  7
  7
  7
 ⋮ 
 18
 18
 18
[torch.LongTensor of size 19713]

In [11]:
final_df_test['predicted_Q_physician'] = predicted_Q_physician.data.numpy()
final_df_test['predicted_Q_optimal'] = predicted_Q_optimal.data.numpy()
final_df_test['best_action'] = best_actions_list.numpy()

In [12]:
final_df_test.to_csv(os.path.join(data_path, 'predicted_Q_df_test.csv'))